In [111]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

BASE_DIR = '../'


In [112]:
import os
import fiona
BASE_DIR = os.path.dirname(os.getcwd())
os.chdir(BASE_DIR + '/src/data/')
import util
import analysis_util
os.chdir(BASE_DIR + '/notebooks/')

In [113]:
def print_summary(total_crashes, crashes, counts, inter_count, non_inter_count):
    """                                                                                                                                                                 
    Info about intersections vs non intersections and their crash rate                                                                                                  
    """
    
    print "Total number of crashes:\t\t\t\t" + str(total_crashes)
    print "Total number of crash locations:\t\t\t\t" + str(len(crashes))
    print "Number of intersections:\t\t\t\t" + str(inter_count)
    print "Number of non-intersections:\t\t\t\t" + str(non_inter_count)
    print "Percentage of segments that are intersections:\t\t" \
        + str(round(100 * float(inter_count)/float(inter_count + non_inter_count)))
    print ""

    print "Number of intersections with at least 1 crash:\t\t" + str(counts['inter']) 
    print 'Number of intersections with more than 1 crash:\t\t' + str(counts['inter_plus'])
    print "Number of non-intersections with at least 1 crash:\t" + str(counts['non_inter'])
    print "Number of non-intersections with more than 1 crash:\t" + str(counts['non_inter_plus'])
    print ""

    total_percent = round(float(100 * (counts['inter'] + counts['non_inter']) / float(inter_count + non_inter_count)))
    print "percent of all segments with crash:\t\t\t" + str(total_percent)

    # Percentage of intersections/non-intersections                                                                                                                     
    # that have at least one crash
    inter_percent = round(float(100 * counts['inter'])/float(inter_count))
    print "percent of intersections with crash:\t\t\t" + str(inter_percent)

    non_inter_percent = round(100 * float(counts['non_inter'])/float(non_inter_count))
    print "percentage of non-intersections with crash:\t\t" + str(non_inter_percent)

    print "percentage of intersections with more than 1 crash:\t" + str(
        round(float(100 * counts['inter_plus'])/float(inter_count)))

    print "percentage of non-intersections with more than 1 crash:\t" + str(
        round(float(100 * counts['non_inter_plus'])/float(non_inter_count)))


In [114]:
def print_concern_summary(results):
    """                                                                                                                                                                 
    What percentage of intersections with concerns had crashes                                                                                                          
    at varying counts of concerns?                                                                                                                                      
    """

    results = results[0:5]

    display(pd.DataFrame(results, columns=[
        '# of concerns at this segment',
        '% of segments w/ v0 complaint w/ crash',
        'total # of segments with this many or more concerns',
        '% of intersections with this many or more concerns with a crash',
        '# of intersections with this many concerns',
        '% of non-intersections with this many concerns with a crash',
        '# of non-intersections with this many concerns',
        '% of these locations that were at an intersection'
    ]))

In [115]:
def print_concerns_by_type(results):
    df = pd.DataFrame(results, columns=[
        'Request type',
#        '# of requests of this type that appear more than once at a segment',
        '% of segments with this type of request with a crash',
        '# of segments with this type of request with a crash',
        '# of segments with this type of a request total',
        '% of intersections with this type of request with a crash',
        '# of intersections with this type of request with a crash',
        '# of intersections with this type of request total',
        '% of non-intersections with this type of request with a crash',
        '# of non-intersections with this type of request with a crash',
        '# of non-intersections with this type of request total',
#        '% of complaint locations that were intersections'
        #'% of intersections with this type of request with a ped crash',
        #'# of ped crashes in intersection with this request',
        #'# of ped crashes in non-intersection with this request'
    ])
    df.sort_values(by="# of segments with this type of a request total", ascending=0)
    display(df)


In [116]:
def print_analysis_for_city(dir, crash_file, concern_file, category_field='REQUESTTYPE', years=None, cutoff=100):
    inters = util.read_segments(os.path.join(dir, 'processed/maps'), get_non_inter=False)
    non_inters = util.read_segments(os.path.join(dir, 'processed/maps'), get_inter=False)
    inter_count = len(inters[0])
    non_inter_count = len(non_inters[0])

    crash_file = os.path.join(dir, 'processed', crash_file)
    concern_file = os.path.join(dir, 'processed', concern_file)
    
    crash_data, crashes = util.group_json_by_location(
        crash_file, years=years, yearfield='CALENDAR_DATE')
    concern_data, concerns = util.group_json_by_location(
        concern_file,
        otherfields=[category_field])

    total_crashes, crash_locations, crash_results = analysis_util.summary_crash_rate(crashes)
   
    print_summary(total_crashes, crashes, crash_results, inter_count, non_inter_count)
    total_count, total_loc, inter_total, inter_loc, non_inter_total, non_inter_loc, concern_summary = analysis_util.summary_concern_counts(
        crashes, concerns)
    results = analysis_util.concern_percentages(concern_summary)
    print_concern_summary(results)
 
    concerns_by_type = analysis_util.concern_counts_by_type(
        concern_data, crashes, category_field)
    concern_percent_by_type = analysis_util.concern_percentages_by_type(
        concerns_by_type, cutoff)
    print_concerns_by_type(concern_percent_by_type)


In [117]:
print_analysis_for_city('../osm-data', 'crash_joined.json', 'concern_joined.json', years=[2016])
print "-------------------------------"


Read in 7845 intersection, 0 non-intersection segments
Read in 0 intersection, 15434 non-intersection segments
Total number of crashes:				3202
Total number of crash locations:				1958
Number of intersections:				7845
Number of non-intersections:				15434
Percentage of segments that are intersections:		34.0

Number of intersections with at least 1 crash:		1110
Number of intersections with more than 1 crash:		397
Number of non-intersections with at least 1 crash:	847
Number of non-intersections with more than 1 crash:	152

percent of all segments with crash:			8.0
percent of intersections with crash:			14.0
percentage of non-intersections with crash:		5.0
percentage of intersections with more than 1 crash:	5.0
percentage of non-intersections with more than 1 crash:	1.0


,# of concerns at this segment,% of segments w/ v0 complaint w/ crash,total # of segments with this many or more concerns,% of intersections with this many or more concerns with a crash,# of intersections with this many concerns,% of non-intersections with this many concerns with a crash,# of non-intersections with this many concerns,% of these locations that were at an intersection
0,1,20.0,4345,30.0,1981,11.0,2364,46.0
1,2,29.0,1596,42.0,814,15.0,782,51.0
2,3,36.0,804,51.0,430,19.0,374,53.0
3,4,41.0,474,52.0,271,27.0,203,57.0
4,5,49.0,308,62.0,181,31.0,127,59.0


[[u'people speed', 15.0, 151, 1018, 26.0, 81, 312, 10.0, 70, 706], [u'it\u2019s hard to see / low visibility', 24.0, 95, 391, 30.0, 81, 271, 12.0, 14, 120], [u"there's not enough time to cross the street", 48.0, 71, 149, 57.0, 63, 110, 21.0, 8, 39], [u'there are no bike facilities or they need maintenance', 24.0, 32, 134, 57.0, 16, 28, 15.0, 16, 106], [u"bike facilities don't exist or need improvement", 24.0, 227, 965, 42.0, 102, 245, 17.0, 125, 720], [u'people double park their vehicles', 20.0, 91, 463, 33.0, 46, 140, 14.0, 45, 323], [u"people don't yield while going straight", 29.0, 80, 278, 32.0, 64, 199, 20.0, 16, 79], [u'people cross away from the crosswalks', 29.0, 71, 247, 38.0, 48, 126, 19.0, 23, 121], [u"people don't yield while turning", 40.0, 196, 494, 47.0, 182, 385, 13.0, 14, 109], [u'the roadway surface needs improvement', 30.0, 67, 227, 46.0, 39, 85, 20.0, 28, 142], [u"sidewalks/ramps don't exist or need improvement", 23.0, 64, 283, 34.0, 39, 114, 15.0, 25, 169], [u'peop

,Request type,% of segments with this type of request with a crash,# of segments with this type of request with a crash,# of segments with this type of a request total,% of intersections with this type of request with a crash,# of intersections with this type of request with a crash,# of intersections with this type of request total,% of non-intersections with this type of request with a crash,# of non-intersections with this type of request with a crash,# of non-intersections with this type of request total
0,people speed,15.0,151,1018,26.0,81,312,10.0,70,706
1,it’s hard to see / low visibility,24.0,95,391,30.0,81,271,12.0,14,120
2,there's not enough time to cross the street,48.0,71,149,57.0,63,110,21.0,8,39
3,there are no bike facilities or they need main...,24.0,32,134,57.0,16,28,15.0,16,106
4,bike facilities don't exist or need improvement,24.0,227,965,42.0,102,245,17.0,125,720
5,people double park their vehicles,20.0,91,463,33.0,46,140,14.0,45,323
6,people don't yield while going straight,29.0,80,278,32.0,64,199,20.0,16,79
7,people cross away from the crosswalks,29.0,71,247,38.0,48,126,19.0,23,121
8,people don't yield while turning,40.0,196,494,47.0,182,385,13.0,14,109
9,the roadway surface needs improvement,30.0,67,227,46.0,39,85,20.0,28,142
